In [80]:
GROUP = 0

# Imports

In [ ]:
from pathlib import Path
import re
import os
import glob
from pathlib import Path
import json
from shutil import copyfile
import itertools
from collections import Counter
import pandas as pd
import numpy as np

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

import spacy

from spacy import displacy
from spacy.pipeline import EntityRuler
from spacy.matcher import PhraseMatcher
from spacy.tokens import Doc, Span, Token
from spacy.attrs import intify_attrs

def display_ent(doc):
    displacy.render(doc, style="ent", jupyter=True)
        
base_path = Path('..')
events_path = base_path / 'events'
dictionary_path = base_path / 'dictionary'
patterns_path = dictionary_path / 'patterns'
group_events_path = events_path / f'group_{GROUP}_events.csv'
labelled_path = events_path / f'group_{GROUP}_labelled.csv'

# Load Spacy NLP with Custom Patterns

In [2]:
patterns = []
for filename in patterns_path.glob("*.json"):
    with open(filename, encoding="utf8") as f:
        patterns += json.load(f)
        
nlp = spacy.load("en_core_web_lg")
ruler = EntityRuler(nlp, overwrite_ents=True)
ruler.add_patterns(patterns)
nlp.add_pipe(ruler)

# Load DataFrame

In [3]:
if labelled_path.is_file():
    df = pd.read_csv(labelled_path)
elif group_events_path.is_file():        
    df = pd.read_csv(group_events_path)
    df['reviewed'] = False
else:
    print(f'{group_events_path} does not exist')
    
cols = [c for c in df.columns if not c.startswith('Unnamed')]
df = df[cols]

# Labelling

In [82]:
df_to_review = df.loc[~df.reviewed]
n = len(df)
idx = df_to_review.index[0]

def get_index_text(idx):
    return f"<h3>Current Index: {idx} / {n}</h3>"

def get_trigger_text(idx):  
    trigger_words = ''.join([f'<li>{w}</li>' for w in df.iloc[idx]['trigger_words_in_event'].split(',')])
    return f"<h3>Trigger Words:</h3><ul>{trigger_words}</ul>"

def yes_pressed(b):
    button_pressed(label=True)

def no_pressed(b):
    button_pressed(label=False)

def button_pressed(label):
    global idx
    
    df.loc[idx, 'reviewed'] = True
    df.loc[idx,'Near Miss Event'] = label
    df.loc[idx, 'Key trigger phrase'] = key_triggers.value
    key_triggers.value = ''
    
    idx += 1
    idx_text.value = get_index_text(idx)
    triggers_widget.value = get_trigger_text(idx)
    
    with event_text_output:
        event_text_output.clear_output()
        display_ent(nlp(df.iloc[idx].event_text))
        
def save_pressed(b):
    df.to_csv(labelled_path, index=False)

idx_text = widgets.HTML(value=get_index_text(idx))
triggers_widget = widgets.HTML(value=get_trigger_text(idx))

event_text_output = widgets.Output()

yes = widgets.Button(description='Near Miss Event', button_style='success')
yes.on_click(yes_pressed)

no = widgets.Button(description='Not Near Miss Event', button_style='danger')
no.on_click(no_pressed)

save = widgets.Button(description='Save DataFrame', button_style='primary')
save.on_click(save_pressed)

label_buttons = widgets.HBox([yes, no])
buttons = widgets.VBox([label_buttons, save])

key_triggers = widgets.Textarea(description='Key Triggers:')

with event_text_output:
    display_ent(nlp(df.iloc[idx].event_text))
                            
display(idx_text, triggers_widget, event_text_output, key_triggers, buttons)

HTML(value='<h3>Current Index: 285 / 530</h3>')

HTML(value='<h3>Trigger Words:</h3><ul><li>mineralisation</li></ul>')

Output()

Textarea(value='', description='Key Triggers:')